In [23]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
import sys

数据收集 数据划分 

数据读取(DataLoader)

Dataloder:

1.Sampler  -->index

2.Dataset  -->img label

数据预处理(transforms)

torch.utils.data.Dataloader 构建可迭代的数据装载器 params：

dataset： dataset类 决定数据从哪儿读取以及如何读取

batchsize：批大小

num_works: 是否多进程读取数据

shuffle：每个epoch是否乱序

drop_last ：当样本数不能被batchsize整除时 是否舍弃最后一批数据

torch.utils.data.Dataset  Dataset抽象类 所有自定义的dataset需要继承他 并且复写__getitem__()  params；

getitem:接收一个索引 返回一个样本

数据读取：

1.读哪些数据

2.从哪读数据

3.怎么读数据

In [4]:
base_dir = os.path.dirname(os.path.abspath('DataLoaderAndDataset.ipynb'))
base_dir

'd:\\Pytorch_Study\\ch02'

In [5]:
def makedir(newdir):
    if not os.path.exists(newdir):
        os.makedirs(newdir)

In [16]:
dataset_dir = os.path.abspath(os.path.join(base_dir,'..','data','RMB_data'))
split_dir = os.path.abspath(os.path.join(base_dir,'..','data','RMB_split_new'))
train_dir = os.path.join(split_dir,'train')
valid_dir = os.path.join(split_dir,'valid')
test_dir = os.path.join(split_dir,'test')

if not os.path.exists(dataset_dir):
    raise Exception('lack data')

train_pct = 0.8
vaild_pct = 0.1
test_pct = 0.1

os.walk()函数来遍历一个目录及其子目录中的所有文件和文件夹 params:

root：表示当前正在遍历的目录的路径

dirs：一个列表，包含当前目录中的子文件夹的名称

files：一个列表，包含当前目录中的文件的名称

这样，通过os.walk(dataset_dir)，你可以遍历dataset_dir目录及其所有子目录中的文件和文件夹，并使用root、dirs和files来处理它们

In [20]:
for root,dirs,files in os.walk(dataset_dir):
    #获取子文件夹中所有文件的列表：
    #imgs = os.listdir(os.path.join(root, sub_dir))，这将返回子文件夹内所有文件的名称列表
    for sub_dir in dirs:
        print(sub_dir)
        imgs = os.listdir(os.path.join(root,sub_dir))
        print(imgs,type(imgs),len(imgs))

1
['01B68AKT.jpg', '01EIM65B.jpg', '01LNYXO4.jpg', '01MF2W5S.jpg', '01NISKCG.jpg', '02C4V1SW.jpg', '03WGM2XG.jpg', '03WV6GFZ.jpg', '049I6MVB.jpg', '04A32I57.jpg', '04MGL637.jpg', '04QE2KHA.jpg', '04QGLB16.jpg', '04RWK2B5.jpg', '04VRAHK2.jpg', '04YVW9CN.jpg', '059GS728.jpg', '05MLGSGI.jpg', '05MO1N93.jpg', '067TZA8C.jpg', '069N3OK2.jpg', '073LW92O.jpg', '07GVXBMG.jpg', '07HTXU3W.jpg', '07IUEGQX.jpg', '07PVUGTB.jpg', '07R6PKIX.jpg', '08596RNG.jpg', '08C3EHPG.jpg', '09F2SGOT.jpg', '09PUM1HY.jpg', '0B89KOA3.jpg', '0BGHNV6P.jpg', '0BRO7XVG.jpg', '0C4UDH9S.jpg', '0CNU427V.jpg', '0CTO7MER.jpg', '0D29EFZO.jpg', '0D6HCAXL.jpg', '0D73KYGN.jpg', '0DLF8NU6.jpg', '0DLW9G7O.jpg', '0DRZXTK3.jpg', '0E4QRCTS.jpg', '0E5Q62TM.jpg', '0E6AGCOW.jpg', '0EBSK2GF.jpg', '0EMSWVIR.jpg', '0EP9R4N8.jpg', '0EZ7ND18.jpg', '0F9CEKGH.jpg', '0FA8LD9Z.jpg', '0FY3IOKC.jpg', '0G7ZDUOL.jpg', '0GE1UZT5.jpg', '0GFISMAH.jpg', '0GHKAWQX.jpg', '0GP1SABX.jpg', '0GPYRDQM.jpg', '0GQBKCAW.jpg', '0GRZFSDG.jpg', '0GZN8V26.jpg', '0H2X

当你有一个列表需要从中筛选出符合特定条件的元素时，你可以使用Python中的filter()函数。

filter()函数接受一个函数和一个可迭代对象（如列表），并返回一个新的可迭代对象，其中包含符合条件的元素。

在这段代码中，imgs 是一个列表，其中包含了文件名的字符串。lambda x: x.endswith('.jpg') 是一个匿名函数，它接受一个参数 x（这里指的是每个文件名），然后检查这个文件名是否以 .jpg 结尾。如果是，返回 True，否则返回 False。

filter() 函数会对列表中的每个元素应用这个条件函数，并保留那些满足条件（返回 True）的元素，最终构成一个新的列表。

所以，这行代码的作用就是从 imgs 列表中筛选出仅包含 .jpg 扩展名的文件名，然后构建一个只包含这些文件名的新列表。这就实现了将原始列表中非 .jpg 文件名剔除的操作。

In [24]:
for root,dirs,files in os.walk(dataset_dir):
    for sub_dir in dirs:
        imgs = os.listdir(os.path.join(root,sub_dir))
        imgs = list(filter(lambda x:x.endswith('.jpg'),imgs))
        random.shuffle(imgs)
        img_count = len(imgs)

        train_point = int(img_count*train_pct)
        valid_point = int(img_count*(vaild_pct+train_pct))

        if img_count == 0:
            print('lack data')
            sys.exit(0)

        for i in range(img_count):
            if i < train_point:
                out_dir = os.path.join(train_dir,sub_dir)
            elif i < valid_point:
                out_dir = os.path.join(valid_dir,sub_dir)
            else:
                out_dir = os.path.join(test_dir,sub_dir)

            makedir(out_dir)

            target_path = os.path.join(out_dir,imgs[i])
            src_path = os.path.join(dataset_dir,sub_dir,imgs[i])

            shutil.copy(src_path,target_path)

            print('class{} train{} valid{} test{}'.format(sub_dir,train_point,valid_point-train_point,img_count-valid_point))

            print('已在 {} 创建好划分的数据'.format(out_dir))

            

class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data\RMB_split_new\train\1 创建好划分的数据
class1 train80 valid10 test10
已在 d:\Pytorch_Study\data